In [83]:
from dataclasses import dataclass
from typing import Annotated, Any, Callable, TypeVar, Dict, Tuple

from autogen import register_function
from autogen.agentchat import ConversableAgent
from autogen.io import IOStream

from fastagency.core import ChatMessage, Chatable, ConsoleIO, MultipleChoice, Workflow, Workflows
# from fastagency.core.autogen import IOStreamAdapter

from fixtures import openai_gpt4o_mini_llm_config, openai_gpt4o_llm_config

In [84]:
class IOStreamAdapter:  # IOStream
    def __init__(self, io: Chatable) -> None:
        """Initialize the adapter with a ChatableIO object.

        Args:
            io (ChatableIO): The ChatableIO object to adapt

        """
        self.io = io

    def print(
        self, *objects: Any, sep: str = " ", end: str = "\n", flush: bool = False
    ) -> None:
        body = sep.join(map(str, objects)) + end
        message = ChatMessage(sender=None, recepient=None, heading=None, body=body)
        self.io.print(message)

    def input(self, prompt: str = "", *, password: bool = False) -> str:
        message = ChatMessage(sender=None, recepient=None, heading=None, body=prompt)
        return self.io.input(message, password=password)

In [85]:
class AutoGenWorkflows(Workflows):
    def __init__(self):
        self._workflows: Dict[str, Tuple[Callable[[Chatable, str, str], str], str]] = {}

    def register(self, name: str, description: str) -> Callable[[Workflow], Workflow]:
        def decorator(func: Workflow) -> Workflow:
            if name in self._workflows:
                raise ValueError(f"A workflow with name '{name}' already exists.")
            
            self._workflows[name] = func, description
            return func
        
        return decorator

    def run(self, name: str, session_id: str, io: Chatable, initial_message: str) -> str:
        workflow, description = self._workflows[name]

        # def ask_user(message: Annotated[str, "message for the user"]) -> str:
        #     msg = ChatMessage(
        #         sender="student",
        #         recepient="user",
        #         heading="User info needed:",
        #         body=message
        #     )
        #     return io.input(msg)
        
        # def report_to_user(message: Annotated[str, "message for the user"]) -> str:
        #     msg = ChatMessage(
        #         sender="student",
        #         recepient="user",
        #         heading="Report:",
        #         body=message
        #     )
        #     io.print(msg)
        #     return "ok"
        
        iostream = IOStreamAdapter(io)
        
        with IOStream.set_default(iostream):
            return workflow(io=io, initial_message=initial_message, session_id=session_id)
    

In [87]:
wf = AutoGenWorkflows()

@wf.register(name="simple_learning", description="Student and teacher learning chat")
def simple_workflow(io: Chatable, initial_message: str, session_id: str) -> str:
    student_agent = ConversableAgent(
        name="Student_Agent",
        system_message="You are a student willing to learn.",
        llm_config=openai_gpt4o_mini_llm_config,
    )
    teacher_agent = ConversableAgent(
        name="Teacher_Agent",
        system_message="You are a math teacher.",
        llm_config=openai_gpt4o_mini_llm_config,
    )

    chat_result = student_agent.initiate_chat(
        teacher_agent,
        message=initial_message,
        summary_method="reflection_with_llm",
        max_turns=5,
    )

    return chat_result.summary

initial_message="What is triangle inequality?"

io = ConsoleIO()

io.print(ChatMessage(sender="user", recepient="workflow", heading="Workflow BEGIN", body=f"Starting workflow with initial_message: {initial_message}"))

result = wf.run(name="simple_learning", session_id="session_id", io=io.create_subconversation(), initial_message="What is triangle inequality?")

io.print(ChatMessage(sender="user", recepient="workflow", heading="Workflow END", body=f"Starting workflow with initial_message: {initial_message}"))


AttributeError: 'str' object has no attribute 'sender'

In [ ]:
wf = AutoGenWorkflows()

@wf.register(name="exam_practice", description="Student and teacher chat")
def exam_learning(io: Chatable, initial_message: str, session_id: str) -> str:

    def is_termination_msg(msg: str) -> bool:
        return msg["content"] is not None and "TERMINATE" in msg["content"]

    student_agent = ConversableAgent(
        name="Student_Agent",
        system_message="You are a student writing a practice test. Your task is as follows:\n"
            "  1) Retrieve exam questions by calling a function.\n" 
            "  2) Write a draft of proposed answers and engage in dialogue with your tutor.\n"
            "  3) Once you are done with the dialogue, register the final answers by calling a function.\n"
            "  4) Retrieve the final grade by calling a function.\n"
            "Finally, terminate the chat by saying 'TERMINATE'.",
        llm_config=openai_gpt4o_mini_llm_config,
        human_input_mode="NEVER",
        is_termination_msg=is_termination_msg,
    )
    teacher_agent = ConversableAgent(
        name="Teacher_Agent",
        system_message="You are a teacher.",
        llm_config=openai_gpt4o_mini_llm_config,
        human_input_mode="NEVER",
        is_termination_msg=is_termination_msg,
    )

    def retrieve_exam_questions(message: Annotated[str, "Message for examiner"]) -> str:
        return "1) Mona Lisa 2) Innovations 3) Florence at the time of Leonardo 4) The Last Supper 5) Vit"
        msg = ChatMessage(
            sender="student",
            recepient="teacher",
            heading="Question for examiner",
            body=message
        )
        return io.input(msg)
    
    def write_final_answers(message: Annotated[str, "Message for examiner"]) -> str:
        msg = ChatMessage(
            sender="student",
            recepient="teacher",
            heading="Final answers",
            body=message
        )
        return io.print(msg)
    
    def get_final_grade(message: Annotated[str, "Message for examiner"]) -> str:
        msg = MultipleChoice(
            message = ChatMessage(
                sender="student",
                recepient="teacher",
                heading="Final grade",
                body=message,
            ),
            choices=["A", "B", "C", "D", "F"],
        )
        return io.print(msg)

    register_function(
        retrieve_exam_questions,
        caller=student_agent,
        executor=teacher_agent,
        name="retrieve_exam_questions",
        description="Get exam questions from examiner",
    )

    register_function(
        write_final_answers,
        caller=student_agent,
        executor=teacher_agent,
        name="write_final_answers",
        description="Write a final answers to exam questions to examiner, but only after discussing with the tutor first.",
    )

    register_function(
        get_final_grade,
        caller=student_agent,
        executor=teacher_agent,
        name="get_final_grade",
        description="Get the final grade after submitting the answers.",
    )

    chat_result = teacher_agent.initiate_chat(
        student_agent,
        message=initial_message,
        summary_method="reflection_with_llm",
        max_turns=10,
    )

    return chat_result.summary

result = wf.run(name="exam_practice", session_id="session_id", io=ConsoleIO(), initial_message="Prepare for the test about Leonardo da Vinci.")

print(result)

In [ ]:
from pprint import pprint

from openai import OpenAI
from autogen.oai.client import OpenAIClient

openai_config={'api_key': openai_gpt4o_mini_llm_config["config_list"][0]["api_key"]}
client = OpenAIClient(OpenAI(**openai_config))

params={'messages': [{'content': "You are a student writing a practice test. Your task is as follows:\n  1) Retrieve exam questions by calling a function.\n  2) Write a draft of proposed answers and engage in dialogue with your tutor.\n  3) Once you are done with the dialogue, register the final answers by calling a function.\n  4) Retrieve the final grade by calling a function.\nFinally, terminate the chat by saying 'TERMINATE'.", 'role': 'system'}, {'content': 'Prepare for the test about Leonardo da Vinci.', 'role': 'user', 'name': 'Teacher_Agent'}, {'tool_calls': [{'id': 'call_WtPX2rq3saIIqk4hM4jjAhNY', 'function': {'arguments': '{"message":"Please provide the exam questions for the test about Leonardo da Vinci."}', 'name': 'retrieve_exam_questions'}, 'type': 'function'}], 'content': None, 'role': 'assistant'}, {'tool_call_id': 'call_WtPX2rq3saIIqk4hM4jjAhNY', 'role': 'tool', 'content': '1) Mona Lisa 2) Innovations 3) Florence at the time of Leonardo 4) The Last Supper 5) Vit'}, {'content': "I have retrieved the exam questions about Leonardo da Vinci. Here they are:\n\n1. Mona Lisa\n2. Innovations\n3. Florence at the time of Leonardo\n4. The Last Supper\n5. Vitruvian Man\n\nNow, I'll draft proposed answers for each question. Let's start with the first one: \n\n1. **Mona Lisa**: The Mona Lisa, painted by Leonardo da Vinci in the early 16th century, is arguably the most famous painting in the world. It is known for the subject's enigmatic expression and da Vinci's masterful use of sfumato, which creates a soft transition between colors and tones.\n\nWhat do you think of this draft answer? Would you like to add or change anything?", 'role': 'assistant', 'name': 'Student_Agent'}, {'content': "Your draft answer for the Mona Lisa is well-articulated and captures the essence of the painting and its significance. Here are a few suggestions to enhance your response:\n\n1. **Historical Context**: You could mention that the painting was created between 1503 and 1506 and is housed in the Louvre Museum in Paris.\n   \n2. **Artistic Techniques**: You might want to elaborate on the technique of sfumato, explaining how it contributes to the depth and realism of the portrait.\n\n3. **Cultural Impact**: It could be beneficial to touch on the painting's influence on art and culture, as well as its status as a symbol of the Renaissance.\n\nHere's a revised version incorporating these points:\n\n**Revised Answer**: The Mona Lisa, painted by Leonardo da Vinci between 1503 and 1506, is arguably the most famous painting in the world and is housed in the Louvre Museum in Paris. It is renowned for the subject's enigmatic expression, which has intrigued viewers for centuries. Da Vinci's masterful use of sfumato, a technique that allows for a soft transition between colors and tones, adds a remarkable depth and realism to the piece. The Mona Lisa has not only influenced countless artists and movements but has also become a symbol of the Renaissance and a cultural icon in its own right.\n\nLet me know if you would like to move on to the next question or make further adjustments!", 'role': 'user', 'name': 'Teacher_Agent'}], 'temperature': 0.8, 'tools': [{'type': 'function', 'function': {'description': 'Get exam questions from examiner', 'name': 'retrieve_exam_questions', 'parameters': {'type': 'object', 'properties': {'message': {'type': 'string', 'description': 'Message for examiner'}}, 'required': ['message'], 'additionalProperties': False}}}, {'type': 'function', 'function': {'description': 'Write a final answers to exam questions to examiner, but only after discussing with the tutor first.', 'name': 'write_final_answers', 'parameters': {'type': 'object', 'properties': {'message': {'type': 'string', 'description': 'Message for examiner'}}, 'required': ['message'], 'additionalProperties': False}}}, {'type': 'function', 'function': {'description': 'Get the final grade after submitting the answers.', 'name': 'get_final_grade', 'parameters': {'type': 'object', 'properties': {'message': {'type': 'string', 'description': 'Message for examiner'}}, 'required': ['message'], 'additionalProperties': False}}}], 'model': 'gpt-4o-mini', 'stream': False}
pprint(params)

In [ ]:
params = {'messages': [{'content': 'You are a student writing a practice test. Your '
                          'task is as follows:\n'
                          '  1) Retrieve exam questions by calling a '
                          'function.\n'
                          '  2) Write a draft of proposed answers and engage '
                          'in dialogue with your tutor.\n'
                          '  3) Once you are done with the dialogue, register '
                          'the final answers by calling a function.\n'
                          '  4) Retrieve the final grade by calling a '
                          'function.\n'
                          "Finally, terminate the chat by saying 'TERMINATE'.",
               'role': 'system'},
              {'content': 'Prepare for the test about Leonardo da Vinci.',
               'name': 'Teacher_Agent',
               'role': 'user'},
              {'content': None,
               'role': 'assistant',
               'tool_calls': [{'function': {'arguments': '{"message":"Please '
                                                         'provide the exam '
                                                         'questions for the '
                                                         'test about Leonardo '
                                                         'da Vinci."}',
                                            'name': 'retrieve_exam_questions'},
                               'id': 'call_WtPX2rq3saIIqk4hM4jjAhNY',
                               'type': 'function'}]},
              {'content': '1) Mona Lisa 2) Innovations 3) Florence at the time '
                          'of Leonardo 4) The Last Supper 5) Vit',
               'role': 'tool',
               'tool_call_id': 'call_WtPX2rq3saIIqk4hM4jjAhNY'},
              {'content': 'I have retrieved the exam questions about Leonardo '
                          'da Vinci. Here they are:\n'
                          '\n'
                          '1. Mona Lisa\n'
                          '2. Innovations\n'
                          '3. Florence at the time of Leonardo\n'
                          '4. The Last Supper\n'
                          '5. Vitruvian Man\n'
                          '\n'
                          "Now, I'll draft proposed answers for each question. "
                          "Let's start with the first one: \n"
                          '\n'
                          '1. **Mona Lisa**: The Mona Lisa, painted by '
                          'Leonardo da Vinci in the early 16th century, is '
                          'arguably the most famous painting in the world. It '
                          "is known for the subject's enigmatic expression and "
                          "da Vinci's masterful use of sfumato, which creates "
                          'a soft transition between colors and tones.\n'
                          '\n'
                          'What do you think of this draft answer? Would you '
                          'like to add or change anything?',
               'name': 'Student_Agent',
               'role': 'assistant'},
              {'content': 'Your draft answer for the Mona Lisa is '
                          'well-articulated and captures the essence of the '
                          'painting and its significance. Here are a few '
                          'suggestions to enhance your response:\n'
                          '\n'
                          '1. **Historical Context**: You could mention that '
                          'the painting was created between 1503 and 1506 and '
                          'is housed in the Louvre Museum in Paris.\n'
                          '   \n'
                          '2. **Artistic Techniques**: You might want to '
                          'elaborate on the technique of sfumato, explaining '
                          'how it contributes to the depth and realism of the '
                          'portrait.\n'
                          '\n'
                          '3. **Cultural Impact**: It could be beneficial to '
                          "touch on the painting's influence on art and "
                          'culture, as well as its status as a symbol of the '
                          'Renaissance.\n'
                          '\n'
                          "Here's a revised version incorporating these "
                          'points:\n'
                          '\n'
                          '**Revised Answer**: The Mona Lisa, painted by '
                          'Leonardo da Vinci between 1503 and 1506, is '
                          'arguably the most famous painting in the world and '
                          'is housed in the Louvre Museum in Paris. It is '
                          "renowned for the subject's enigmatic expression, "
                          'which has intrigued viewers for centuries. Da '
                          "Vinci's masterful use of sfumato, a technique that "
                          'allows for a soft transition between colors and '
                          'tones, adds a remarkable depth and realism to the '
                          'piece. The Mona Lisa has not only influenced '
                          'countless artists and movements but has also become '
                          'a symbol of the Renaissance and a cultural icon in '
                          'its own right.\n'
                          '\n'
                          'Let me know if you would like to move on to the '
                          'next question or make further adjustments! ',
            #    'name': 'Teacher_Agent',
               'role': 'user'}],
 'model': 'gpt-4o-mini',
 'stream': False,
 'temperature': 0.8,
 'tools': [{'type': 'function',
            'function': {'name': 'retrieve_exam_questions',
                         'description': 'Get exam questions from examiner',
                         'parameters': {'type': 'object',
                                        'properties': {'message': {'type': 'string',
                                                                   'description': 'Message '
                                                                                  'for '
                                                                                  'examiner',
                                                                   }},
                                        'required': ['message'],
                                        'additionalProperties': False,
                                        }},
            },
            {'type': 'function',
            'function': {'name': 'write_final_answers',
                         'description': 'Write a final answers to exam '
                                        'questions to examiner, but only after '
                                        'discussing with the tutor first.',
                         'parameters': {'type': 'object',
                                        'properties': {'message': {'type': 'string',
                                                                   'description': 'Message '
                                                                                  'for '
                                                                                  'examiner',
                                                                   }},
                                        'required': ['message'],
                                        'additionalProperties': False,
                                        }},
            },{'type': 'function',
            'function': {'name': 'get_final_grade',
                         'description': 'Get the final grade after submitting '
                                        'the answers.',
                         'parameters': {'type': 'object',
                                        'properties': {'message': {'type': 'string',
                                                                   'description': 'Message '
                                                                                  'for '
                                                                                  'examiner',
                                                                   }},
                                        'required': ['message'],
                                        'additionalProperties': False,
                                        }},
            },
            
            ]}

import json
print(json.dumps(params, indent=2))

In [ ]:
from typing import List


def remove_messages(params: Dict[str, Any], n: int):
    if n == 0:
        return params
    return {k: v if k != "messages" else v[:-(n+1)].copy() for k, v in params.items()}

def remove_names(params: Dict[str, Any]):
    def _remove_names(messages: List[Dict[str, Any]]):
        def __remove_names(message: Dict[str, Any]):
            return {k: v for k, v in message.items() if k != "name"}
        
        return [__remove_names(message) for message in messages]
    return {k: _remove_names(v) if k == "messages" else v for k, v in params.items()}


In [ ]:
new_params = remove_messages(params, 0)
# new_params = remove_names(new_params)
pprint(new_params)

client.create(new_params)

In [ ]:
assert False

In [ ]:
"1) Mona Lisa 2) Innovations 3) Florence at the time of Leonardo 4) The Last Supper 5) Vit"

In [ ]:
import json
from pprint import pprint
from httpx import Response, Request, Client, Headers

content=b'{"messages": [{"content": "You are a student writing a practice test. Your task is as follows:\\n  1) Retrieve exam questions by calling a function.\\n  2) Write a draft of proposed answers and engage in dialogue with your tutor.\\n  3) Once you are done with the dialogue, register the final answers by calling a function.\\n  4) Retrieve the final grade by calling a function.\\nFinally, terminate the chat by saying \'TERMINATE\'.", "role": "system"}, {"content": "Prepare for the test about Leonardo da Vinci.", "role": "user", "name": "Teacher_Agent"}, {"tool_calls": [{"id": "call_WtPX2rq3saIIqk4hM4jjAhNX", "function": {"arguments": "{\\"message\\":\\"Please provide the exam questions for the test about Leonardo da Vinci.\\"}", "name": "retrieve_exam_questions"}, "type": "function"}], "content": null, "role": "assistant"}, {"tool_call_id": "call_WtPX2rq3saIIqk4hM4jjAhNX", "role": "tool", "content": "1) Mona Lisa 2) Innovations 3) Florence at the time of Leonardo 4) The Last Supper 5) Vit"}, {"content": "I have retrieved the exam questions about Leonardo da Vinci. Here they are:\\n\\n1. Mona Lisa\\n2. Innovations\\n3. Florence at the time of Leonardo\\n4. The Last Supper\\n5. Vitruvian Man\\n\\nNow, I\'ll draft proposed answers for each question. Let\'s start with the first one: \\n\\n1. **Mona Lisa**: The Mona Lisa, painted by Leonardo da Vinci in the early 16th century, is arguably the most famous painting in the world. It is known for the subject\'s enigmatic expression and da Vinci\'s masterful use of sfumato, which creates a soft transition between colors and tones.\\n\\nWhat do you think of this draft answer? Would you like to add or change anything?", "role": "assistant", "name": "Student_Agent"}, {"content": "Your draft answer for the Mona Lisa is well-articulated and captures the essence of the painting and its significance. Here are a few suggestions to enhance your response:\\n\\n1. **Historical Context**: You could mention that the painting was created between 1503 and 1506 and is housed in the Louvre Museum in Paris.\\n   \\n2. **Artistic Techniques**: You might want to elaborate on the technique of sfumato, explaining how it contributes to the depth and realism of the portrait.\\n\\n3. **Cultural Impact**: It could be beneficial to touch on the painting\'s influence on art and culture, as well as its status as a symbol of the Renaissance.\\n\\nHere\'s a revised version incorporating these points:\\n\\n**Revised Answer**: The Mona Lisa, painted by Leonardo da Vinci between 1503 and 1506, is arguably the most famous painting in the world and is housed in the Louvre Museum in Paris. It is renowned for the subject\'s enigmatic expression, which has intrigued viewers for centuries. Da Vinci\'s masterful use of sfumato, a technique that allows for a soft transition between colors and tones, adds a remarkable depth and realism to the piece. The Mona Lisa has not only influenced countless artists and movements but has also become a symbol of the Renaissance and a cultural icon in its own right.\\n\\nLet me know if you would like to move on to the next question or make further adjustments!", "role": "user", "name": "Teacher_Agent"}], "model": "gpt-4o-mini", "stream": false, "temperature": 0.8, "tools": [{"type": "function", "function": {"description": "Get exam questions from examiner", "name": "retrieve_exam_questions", "parameters": {"type": "object", "properties": {"message": {"type": "string", "description": "Message for examiner"}}, "required": ["message"], "additionalProperties": false}}}, {"type": "function", "function": {"description": "Write a final answers to exam questions to examiner, but only after discussing with the tutor first.", "name": "write_final_answers", "parameters": {"type": "object", "properties": {"message": {"type": "string", "description": "Message for examiner"}}, "required": ["message"], "additionalProperties": false}}}, {"type": "function", "function": {"description": "Get the final grade after submitting the answers.", "name": "get_final_grade", "parameters": {"type": "object", "properties": {"message": {"type": "string", "description": "Message for examiner"}}, "required": ["message"], "additionalProperties": false}}}]}'
content = json.loads(content)
headers = Headers({'host': 'api.openai.com', 'accept-encoding': 'gzip, deflate', 'connection': 'keep-alive', 'accept': 'application/json', 'content-type': 'application/json', 'user-agent': 'OpenAI/Python 1.42.0', 'x-stainless-lang': 'python', 'x-stainless-package-version': '1.42.0', 'x-stainless-os': 'Linux', 'x-stainless-arch': 'x64', 'x-stainless-runtime': 'CPython', 'x-stainless-runtime-version': '3.10.14', 'authorization': '[secure]', 'x-stainless-async': 'false', 'cookie': '__cf_bm=kj.AZTH0Lm9wDWy8QCmUJ5FyQG.ggWBnCXtRSH5tDFg-1724761542-1.0.1.1-L5wFsT6Tag1RpdJbqVOudqAV3u6ylCqNBsbpEQBHUMN.AP_wW6GQi2WDyC6iL8g3o51.SOvHrbcQVPccqsf9IA; _cfuvid=Ig2J5SIYSViOgwR.fdIySPteP0Dsv25CogT6FmFzMa0-1724761542039-0.0.1.1-604800000', 'content-length': '4190'})
pprint(content)

In [ ]:
pprint(headers)

In [ ]:
client = Client()
client

In [ ]:
request = Request("POST", "https://api.openai.com/v1/engines/gpt-4o-mini/completions", data=content, headers=headers)
request

In [ ]:
client.send(request)